In [ ]:
import simpy
import random

# Parameters
NUM_DRIVERS = 10
SIM_DURATION = 100
RIDER_REQUEST_INTERVAL = 5  # Average time between rider requests
SURGE_THRESHOLD = 0.8  # Ratio of drivers to requests that triggers surge pricing
BASE_FARE = 2.0
PER_MILE_FARE = 1.5

# Environment
env = simpy.Environment()
riders = simpy.Resource(env, capacity=NUM_DRIVERS)

# Events
def rider_generator(env, riders):
    while True:
        yield env.timeout(random.expovariate(1.0 / RIDER_REQUEST_INTERVAL))
        env.process(rider(env, riders))

def rider(env, riders):
    with riders.request() as req:
        yield req
        # Simulate ride duration and distance
        ride_duration = random.uniform(5, 15)
        distance = random.uniform(1, 10)

        # Calculate fare
        fare = BASE_FARE + PER_MILE_FARE * distance

        # Surge pricing
        if riders.count / NUM_DRIVERS < SURGE_THRESHOLD:
            fare *= 1.5  # Apply surge multiplier

        print(f"Ride completed. Fare: ${fare:.2f}")

# Simulation
for i in range(NUM_DRIVERS):
    env.process(rider_generator(env, riders))
env.run(until=SIM_DURATION)
